In [2]:
import numpy as np
import pandas as pd
import json
import requests
import re
import bs4
from bs4 import BeautifulSoup as bs

# Web Scraping from Allrecipes

In order to compile a dataset of recipes for this project, we will first need to use web scraping and build a JSON object.  For this, we will send requests to www.allrecipes.com and parse their recipe cards.

In [3]:
# set how many pages to scrape recipes from
first_page = 1
last_page = 10

In [4]:
# create empty json file to store recipe data
data = []
with open('recipes.json','w') as out_file:
    json.dump(data, out_file, indent=4)

In [67]:
def save_to_json(title, picture, servings, ingredients, method):
    with open('recipes.json') as in_file:
        data = json.load(in_file)
    
    # if this recipe title already exists in the data, do not add it again
    already_exists = False
    for recipe in data:
        if recipe['title'] == title:
            already_exists = True
    
    # if a new recipe, append to the json object and dump back to the file
    if not already_exists:
        print("Saved: {} ".format(title))
        new_recipe = {}
        new_recipe['title'] = title
        new_recipe['picture'] = picture
        new_recipe['ingredients'] = ingredients
        new_recipe['method'] = method

        data.append(new_recipe)
    
        with open('recipes.json', 'w') as out_file:
            json.dump(data, out_file, indent=4)
    else:
        print("Already Exists: {} ".format(title))

Note that allrecipes has two different HTML layouts for their recipe pages, a regular layout and a layout which supports shopping for ingredients directly from the recipe page.  These two layouts have the information we need in different locations, so we need to differentiate them.  If the title element we initially search for is set to 'None', we have to instead look for the elements where they would be in the second (shopper) layout.

In [68]:
for page in range(first_page, last_page + 1):
    # request the main allrecipes page which lists the top recipes
    source = requests.get("https://www.allrecipes.com/recipes?page=" + str(page))
    print("PARSING PAGE {}".format(page))
    doc = bs(source.text,'html.parser')
    
    # find each recipe linked on the main page, and open their links one by one
    recipe_cards = doc.select('a.fixed-recipe-card__title-link')
    for card in recipe_cards:
        recipe_page_source = requests.get(card['href'])        
        recipe_main = bs(recipe_page_source.text,'html.parser')
        
        # search for the title, picture, ingredients, and method elements
        title = recipe_main.select_one('.recipe-summary__h1')
        if title is not None:
            #for ordinary formatting layout (1)
            layout = 1
            picture = recipe_main.select_one('.rec-photo')
            ingredients = recipe_main.select('.recipe-ingred_txt')
            method = recipe_main.select('.recipe-directions__list--item')
            servings = recipe_main.select_one('#metaRecipeServings')
#           prep_time = recipe_main.select('.prepTime__item')
            
#             for item in prep_time:
#                 item_type = item.select_one('time')
#                 item_time = item.select_one('.prepTime__item--time')
                
#                 if item_type is None:
#                     continue

#                 print(item_type)
                        
        # if the title is 'None', then the page must be in the second layout
        else:
            # for shopper formatting layout (2)
            layout = 2
            title = recipe_main.select_one('h1.headline.heading-content')
            picture = recipe_main.select_one('.inner-container > img')
            ingredients = recipe_main.select('span.ingredients-item-name')
            method = recipe_main.select('div.paragraph > p')
            
#             recipe_meta_items = recipe_main.select('div.recipe-meta-item')
#             for item in recipe_meta_items:
#                 header = item.select('div.recipe-meta-item-header')
        
        # compile a list of ingredients for the current recipe
        ingredients_list = []
        for ingredient in ingredients:
            if ingredient.text != 'Add all ingredients to list' and ingredient.text != '':
                ingredients_list.append(ingredient.text.strip())
            
        # compile a list of method instructions for the current recipe
        method_list = []
        for instruction in method:
            method_list.append(instruction.text.strip())

        # save all data for the current recipe to the json file
        save_to_json(title.text, 
                     picture.attrs['src'], 
                     servings['content'],  
                     ingredients_list, 
                     method_list
                    )
        
print("FINISHED PARSING")

PARSING PAGE 1
Already Exists: Shrimp and Zucchini Rice Bowl 
Already Exists: World's Best Lasagna 
Already Exists: Best Chocolate Chip Cookies 
Already Exists: Fluffy Pancakes 
Already Exists: Good Old Fashioned Pancakes 
Saved: Banana Banana Bread 
Saved: Apple Pie by Grandma Ople 
Saved: Delicious Ham and Potato Soup 
Saved: Banana Crumb Muffins 
Saved: To Die For Blueberry Muffins 
Saved: Best Brownies 
Saved: Chicken Pot Pie IX 


KeyboardInterrupt: 

# Data Analysis

Now we have a JSON object which contains recipe information scraped from the web.  We can use this data and perform some analyses on it.

In [69]:
recipes = pd.read_json(r'recipes.json')
recipes.head()

,ingredients,method,picture,title
0,"[2 tablespoons olive oil, divided, 2 cups slic...",[Heat 1 tablespoon oil in a large skillet over...,https://images.media-allrecipes.com/userphotos...,Shrimp and Zucchini Rice Bowl
1,"[1 pound sweet Italian sausage, ¾ pound lean g...","[In a Dutch oven, cook sausage, ground beef, o...",https://imagesvc.meredithcorp.io/v3/mm/image?u...,World's Best Lasagna
2,"[1 cup butter, softened, 1 cup white sugar, 1 ...",[Preheat oven to 350 degrees F (175 degrees C)...,https://imagesvc.meredithcorp.io/v3/mm/image?u...,Best Chocolate Chip Cookies
3,"[3/4 cup milk, 2 tablespoons white vinegar, 1 ...",[Combine milk with vinegar in a medium bowl an...,https://images.media-allrecipes.com/userphotos...,Fluffy Pancakes
4,"[1 1/2 cups all-purpose flour, 3 1/2 teaspoons...","[In a large bowl, sift together the flour, bak...",https://images.media-allrecipes.com/userphotos...,Good Old Fashioned Pancakes


### Analyzing Ingredients

Let's look at the ingredients that compose each recipe a bit more closely.  First, we want to know what the most common ingredients are among our scraped recipes.

In [227]:
all_ingredients_list = []

for row in recipes['ingredients']:
    for ing in row:
        all_ingredients_list.append(ing)

In [228]:
ingredients = pd.DataFrame(all_ingredients_list, columns=['ingredients'])
ingredients['edited'] = ingredients['ingredients']
ingredients.head()

,ingredients,edited
0,1 (1 pound) fillet salmon fillet,1 (1 pound) fillet salmon fillet
1,1/4 cup avocado oil,1/4 cup avocado oil
2,1 teaspoon curry powder,1 teaspoon curry powder
3,salt to taste,salt to taste
4,"1 mango - peeled, seeded, and diced","1 mango - peeled, seeded, and diced"


Now we attempt to clean the ingredients list to get rid of the 'noise' and have a raw list of ingredient names without units, numbers or descriptors.

In [229]:
# lists of common words we want to remove
units = ['gallon','quart','pint','cup','teaspoon','tablespoon','ounce','pound','can','pinch','serving','slice','package','bottle']
descriptors = ['small','medium','large']

# remove common measuring and descriptive words
for word in units + descriptors:
    plural = word+"s"
    ingredients['edited'] = ingredients['edited'].str.replace(' '+plural+' ', ' ')
    ingredients['edited'] = ingredients['edited'].str.replace(' '+word+' ','')

# remove parantheicals
ingredients['edited'] = ingredients['edited'].str.replace(r'\([^()]*\)','')
# remove text after commas and hyphens
ingredients['edited'] = ingredients['edited'].str.partition(',')
ingredients['edited'] = ingredients['edited'].str.partition(',')
# remove non-alphabetical characters
ingredients['edited'] = ingredients['edited'].str.replace('[^a-zA-Z]', ' ')

# edit down extra spaces caused by adjacent removals
ingredients['edited'] = ingredients['edited'].str.strip()

ingredients.head()

,ingredients,edited
0,1 (1 pound) fillet salmon fillet,fillet salmon fillet
1,1/4 cup avocado oil,avocado oil
2,1 teaspoon curry powder,curry powder
3,salt to taste,salt to taste
4,"1 mango - peeled, seeded, and diced",mango peeled


From here we can investigate which ingredients are the most common.  First, we see which edited rows appear most commonly with a call to .value_counts()

In [236]:
ingredients['edited'].value_counts().head(20)

salt                      92
white sugar               91
butter                    86
all purpose flour         81
eggs                      49
vanilla extract           47
vegetable oil             39
milk                      36
baking soda               36
ground black pepper       32
baking powder             28
ground cinnamon           28
water                     27
onion                     27
egg                       24
olive oil                 23
skinless                  22
cloves garlic             20
grated Parmesan cheese    20
cream cheese              18
Name: edited, dtype: int64

This shows us the most common row values.  However, this data has a few flaws.  Primarily, the issue is that we need the entire row to match to be counted together.  For instance, 'sugar' and 'white sugar' are counted seperately in this analysis.

Another approach might be to store the list of all ingredient 'words' in its own dataframe, and perform a value_counts on it to see what the most common non-unit, non-descriptive words are in our recipe ingredients.  This loses some specificity (we lose the distinction between 'white sugar' and 'brown sugar'), but is helpful in other contexts.

In [237]:
ingredient_words_list = []
for row in ingredients['edited']:
    for word in row.split():
        ingredient_words_list.append(word)

ingredient_words = pd.DataFrame(ingredient_words_list,columns=['words'])

In [238]:
ingredient_words['words'].value_counts().head(20)

sugar      135
salt       134
ground     125
pepper     110
white      109
butter     103
flour       92
all         83
purpose     83
cheese      79
powder      76
oil         68
garlic      67
onion       66
baking      65
chopped     60
dried       59
black       58
sauce       58
cream       53
Name: words, dtype: int64